In [ ]:
"""
We used this notebook to create our datasets.
This includes cleaning the raw dataset (substiture ambiguous AA and remove sequences with an X),
clustering and splitting into train, validation and test set as well as generating txt files
from the fasta datasets for finetuning with the Huggingface run_clm.py script
"""

In [1]:
from sklearn.model_selection import train_test_split
from Bio import SeqIO
import os
import subprocess
import shutil
from parse_clusters import parse_clusters
from sequence_cleaning import remove_X, replace_amino_acids

In [ ]:
# Clean: remove sequences with X
dataset = SeqIO.parse("../datasets/InterProUniprotPF03272.fasta", "fasta")
dataset_xremoved = remove_X(dataset)
with open("../datasets/InterProUniprotPF03272_Xremoved.fasta", 'w') as f:
        SeqIO.write(dataset_xremoved, f, "fasta")

In [ ]:
# Cluster the dataset
devnull = open(os.devnull, 'w')
subprocess.call(['bash', './clustering.sh'], stdout=devnull)

In [ ]:
# Split into train/validation/test
clusters = parse_clusters("../datasets/clustered/InterProUniprotPF03272_all_seqs.fasta")
representatives = [c.members[0] for c in clusters]

random_seed = 123456

train_val_set, test_set = train_test_split(representatives, test_size=0.1, shuffle=True, random_state=random_seed)
train_set, validation_set = train_test_split(train_val_set, test_size=1/9, shuffle=True, random_state=random_seed)

In [ ]:
# Replace ambigous AA
prepared_train_set = replace_amino_acids(random_seed, train_set)
prepared_validation_set = replace_amino_acids(random_seed, validation_set)
prepared_test_set = replace_amino_acids(random_seed, test_set)

In [ ]:
# Save splits to separate files
with open("../datasets/InterProUniprotPF03272prepared_train.fasta", "w") as handle:
    SeqIO.write(prepared_train_set, handle, "fasta")

with open("../datasets/InterProUniprotPF03272prepared_validation.fasta", "w") as handle:
    SeqIO.write(prepared_validation_set, handle, "fasta")

with open("../datasets/InterProUniprotPF03272prepared_test.fasta", "w") as handle:
    SeqIO.write(prepared_test_set, handle, "fasta")

In [ ]:
# Cleanup
shutil.rmtree("../datasets/clustered")
shutil.rmtree("./tmp")

In [12]:
# Generate txt datasets, following the format that the RITA authors specify (https://github.com/lightonai/RITA/issues/10)
records_train_seq = list(SeqIO.parse("../datasets/InterProUniprotPF03272prepared_train.fasta", "fasta"))
records_val_seq = list(SeqIO.parse("../datasets/InterProUniprotPF03272prepared_validation.fasta", "fasta"))

In [10]:
eos_token_rita = "<EOS>"

In [18]:
text_train = eos_token_rita.join([str(record.seq) for record in records_train_seq])
with open("../datasets/InterProUniprotPF03272prepared_train.txt", "w") as handle:
    handle.write(text_train)

text_val = eos_token_rita.join([str(record.seq) for record in records_val_seq])
with open("../datasets/InterProUniprotPF03272prepared_validation.txt", "w") as handle:
    handle.write(text_val)